In [1]:
import numpy as np
import cv2
import time 
import random
from keras.models import load_model

model = load_model(r'C:\Users\HP\Documents\PCV\Poker2.h5')
camera = cv2.VideoCapture(1)
camera.set(10, 500)



RANK_WIDTH = 70
RANK_HEIGHT = 125

SUIT_WIDTH = 70
SUIT_HEIGHT = 100

BKG_THRESH = 70
CARD_THRESH = 40

CORNER_WIDTH = 32
CORNER_HEIGHT = 84

GAME_STATUS = "Playing"
WINNER = None
game_ended = False

card_min_area = 25000  
card_max_area = 45000  
Card = None
KartuTerdeteksi = []

NilaiKartu = {
    "2 Hearts": 2, "3 Hearts": 3, "4 Hearts": 4, "5 Hearts": 5, "6 Hearts": 6, "7 Hearts": 7, "8 Hearts": 8, "9 Hearts": 9, "10 Hearts": 10, 
    "Jack Hearts": 10, "Queen Hearts": 10, "King Hearts": 10, "Ace Hearts": 11, "2 Spades": 2, "3 Spades": 3, "4 Spades": 4, "5 Spades": 5, 
    "6 Spades" : 6, "7 Spades" : 7, "8 Spades" : 8, "9 Spades" : 9, "10 Spades" : 10, "Jack Spades" : 10, "Queen Spades" : 10, "King Spades" : 10, 
    "Ace Spades" : 11, "2 Diamonds" : 2, "3 Diamonds" : 3, "4 Diamonds" : 4, "5 Diamonds" : 5, "6 Diamonds" : 6, "7 Diamonds" : 7, "8 Diamonds" : 8,
    "9 Diamonds" : 9, "10 Diamonds" : 10, "Jack Diamonds" : 10, "Queen Diamonds" : 10, "King Diamonds" : 10, "Ace Diamonds" : 11, "2 Clubs" : 2, 
    "3 Clubs" : 3, "4 Clubs" : 4, "5 Clubs" : 5, "6 Clubs" : 6, "7 Clubs" : 7, "8 Clubs" : 8, "9 Clubs" : 9, "10 Clubs" : 10, "Jack Clubs" :10,
      "Queen Clubs" : 10, "King Clubs" : 10, "Ace Clubs" : 11
}

KartuTerdeteksi_left = []
KartuTerdeteksi_right = []
player_turn_taken = False  # Flag to track if the player has taken their turn
JumlahKartuPemain = 0
JumlahKartuKomputer = 0

class Kartu:

    def __init__(self):
        self.contour = [] # Contour of card
        self.width, self.height = 0, 0 # Width and height of card
        self.corner_pts = [] # Corner points of card
        self.center = [] # Center point of card
        self.warp = [] # 200x300, flattened, grayed, blurred image
        self.rank_img = [] # Thresholded, sized image of card's rank
        self.suit_img = [] # Thresholded, sized image of card's suit
        self.best_rank_match = "Unknown" # Best matched rank
        self.best_suit_match = "Unknown" # Best matched suit
        self.rank_diff = 0 # Difference between rank image and best matched train rank image
        self.suit_diff = 0 # Difference between suit image and best matched train suit image
def ProsesFrame(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    img_w, img_h = np.shape(image)[:2]
    bkg_level = gray[int(img_h/100)][int(img_w/2)]
    thresh_level = bkg_level + BKG_THRESH

    _, thresh = cv2.threshold(blur, thresh_level, 255, cv2.THRESH_BINARY)

    edges = cv2.Canny(blur, threshold1=50, threshold2=150)

    # Combine threshold and edges (optional: based on your use case)
    combined = cv2.bitwise_or(thresh, edges)

    return combined
def ProsesKartu(contour, image):

    Card = Kartu()
    Card.contour = contour

    # Find perimeter of card and use it to approximate corner points
    peri = cv2.arcLength(contour,True)
    approx = cv2.approxPolyDP(contour,0.01*peri,True)

    if len(approx) != 4:
        return None
    
    pts = np.float32(approx)
    Card.corner_pts = pts

    # Find width and height of card's bounding rectangle
    x,y,w,h = cv2.boundingRect(contour)
    Card.width, Card.height = w, h

    # Warp card into 200x300 flattened image using perspective transform
    Card.warp = flattener(image, pts, w, h)
    return Card

def flattener(image, pts, w, h):
    s = np.sum(pts, axis = 2)
    diff = np.diff(pts, axis = -1)
    rect = np.zeros((4, 2), dtype = "float32")

    rect[0] = pts[np.argmin(s)]        # Top-left point
    rect[2] = pts[np.argmax(s)]        # Bottom-right point
    rect[1] = pts[np.argmin(diff)]     # Top-right point
    rect[3] = pts[np.argmax(diff)]     # Bottom-left point

    # Perspective transformation
    maxWidth, maxHeight = 200, 300
    dst = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warp = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
        

    return warp

def ProsesInputModel(image):
    processed_image = cv2.resize(image, (200, 300))
    # if len(processed_image.shape) == 2:  
    #     processed_image = cv2.cvtColor(processed_image, cv2.COLOR_GRAY2BGR)
        
    rgb = cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB)
    processed_image = np.asarray(rgb) / 255.0
    # processed_image = processed_image.astype('float32')
    return processed_image, rgb

def save_flattened_image(image, filename):
    if image is not None:
        cv2.imwrite(filename, image)

def DeteksiLeft(frame):
    global Card
    detected_cards_left = []
    frame_left = frame[:, :frame.shape[1]//2]
    combined_left = ProsesFrame(frame_left)
    contours, _ = cv2.findContours(combined_left, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        area = cv2.contourArea(contour)
        if card_min_area < area < card_max_area:
            
            cv2.drawContours(frame_left, [contour], 0, (255, 0, 0), 2)
            Card = ProsesKartu(contour, frame_left)
            
            if Card is not None:
                processed_card_image, rgb = ProsesInputModel(Card.warp)
                prediction = model.predict(np.expand_dims(processed_card_image, axis=0))
                predicted_class = np.argmax(prediction, axis=1)

                # cv2.imshow('Flattened', Card.warp)

                LabelKartu = [  "2 Hearts", "3 Hearts", "4 Hearts", "5 Hearts", "6 Hearts", "7 Hearts", "8 Hearts", "9 Hearts", "10 Hearts", "Jack Hearts", "Queen Hearts", "King Hearts", "Ace Hearts",
                                "2 Spades", "3 Spades", "4 Spades", "5 Spades", "6 Spades", "7 Spades", "8 Spades", "9 Spades", "10 Spades", "Jack Spades", "Queen Spades", "King Spades", "Ace Spades",
                                "2 Diamonds", "3 Diamonds", "4 Diamonds", "5 Diamonds", "6 Diamonds", "7 Diamonds", "8 Diamonds", "9 Diamonds", "10 Diamonds", "Jack Diamonds", "Queen Diamonds", "King Diamonds", "Ace Diamonds",
                                "2 Clubs", "3 Clubs", "4 Clubs", "5 Clubs", "6 Clubs", "7 Clubs", "8 Clubs", "9 Clubs", "10 Clubs", "Jack Clubs", "Queen Clubs", "King Clubs", "Ace Clubs"]
                LabelPred = LabelKartu[predicted_class[0]]
                card_value = NilaiKartu.get(LabelPred, 0)  
                detected_cards_left.append((LabelPred, card_value))

                x,y,w,h = cv2.boundingRect(contour)
                cv2.rectangle(frame,(x,y),(x+w, y+h),(0,255,0),2)
                cv2.putText(frame,LabelPred,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5, (0,255,0),2)
    
    return frame_left, detected_cards_left

def DeteksiRight(frame):
    global Card
    detected_cards_right = []
    frame_right = frame[:, frame.shape[1]//2:]
    combined_right = ProsesFrame(frame_right)
    contours, _ = cv2.findContours(combined_right, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        area = cv2.contourArea(contour)
        if card_min_area < area < card_max_area:
            x,y,w,h = cv2.boundingRect(contour)
            x_adjusted = x + frame.shape[1]//2 
            cv2.drawContours(frame_right, [contour], 0, (255, 0, 0), 2)
            Card = ProsesKartu(contour, frame_right)
            
            if Card is not None:
                processed_card_image, rgb = ProsesInputModel(Card.warp)
                prediction = model.predict(np.expand_dims(processed_card_image, axis=0))
                predicted_class = np.argmax(prediction, axis=1)

                # cv2.imshow('Flattened', Card.warp)

                LabelKartu = [  "2 Hearts", "3 Hearts", "4 Hearts", "5 Hearts", "6 Hearts", "7 Hearts", "8 Hearts", "9 Hearts", "10 Hearts", "Jack Hearts", "Queen Hearts", "King Hearts", "Ace Hearts",
                                "2 Spades", "3 Spades", "4 Spades", "5 Spades", "6 Spades", "7 Spades", "8 Spades", "9 Spades", "10 Spades", "Jack Spades", "Queen Spades", "King Spades", "Ace Spades",
                                "2 Diamonds", "3 Diamonds", "4 Diamonds", "5 Diamonds", "6 Diamonds", "7 Diamonds", "8 Diamonds", "9 Diamonds", "10 Diamonds", "Jack Diamonds", "Queen Diamonds", "King Diamonds", "Ace Diamonds",
                                "2 Clubs", "3 Clubs", "4 Clubs", "5 Clubs", "6 Clubs", "7 Clubs", "8 Clubs", "9 Clubs", "10 Clubs", "Jack Clubs", "Queen Clubs", "King Clubs", "Ace Clubs"]
                LabelPred = LabelKartu[predicted_class[0]]
                card_value = NilaiKartu.get(LabelPred, 0)
                detected_cards_right.append((LabelPred, card_value))
                cv2.rectangle(frame, (x_adjusted, y), (x_adjusted + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, LabelPred, (x_adjusted, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame_right, detected_cards_right

def play_blackjack(NilaiKiri, JumlahKanan, JumlahKartuPemain, JumlahKartuKomputer, game_status):
    winner = None
    if game_status == "Playing":
        if JumlahKartuPemain >= 2 and JumlahKartuKomputer >= 2:
            if NilaiKiri == 21:
                winner = "Left Player Wins"
                game_status = "Finished"
            elif JumlahKanan == 21:
                winner = "Right Player Wins"
                game_status = "Finished"
            else:
                if abs(21 - NilaiKiri) < abs(21 - JumlahKanan):
                    winner = "Left Player Wins"
                elif abs(21 - NilaiKiri) > abs(21 - JumlahKanan):
                    winner = "Right Player Wins"
                else:
                    winner = "It's a Tie"
                game_status = "Finished"
                
        if game_status == "Playing":
            if NilaiKiri > 21:
                winner = "Right Player Wins"
                game_status = "Finished"
            elif JumlahKanan > 21:
                winner = "Left Player Wins"
                game_status = "Finished"

    return game_status, winner


def get_random_card():
    # List of all possible cards
    all_cards = [card for card in NilaiKartu.keys()]
    # Randomly select a card
    selected_card = random.choice(all_cards)
    card_value = NilaiKartu[selected_card]
    return selected_card, card_value

def computer_play(player_turn_taken):
    global JumlahKartuKomputer 
    if player_turn_taken:
        JumlahKanan = sum(value for _, value in KartuTerdeteksi_right)
        if JumlahKanan < 17:  # AI's strategy
            new_card = get_random_card()
            KartuTerdeteksi_right.append(new_card)
            JumlahKartuKomputer += 1
            return True  # Indicates the AI took its turn
    return False  # Indicates the AI did not take its turn



while camera.isOpened():
    ret, frame = camera.read()
    if not ret:
        continue
    frame_left, new_cards_left = DeteksiLeft(frame)
    frame_right, new_cards_right = DeteksiRight(frame)
    frame_left = cv2.resize(frame_left, (400, 600))
    frame_right = cv2.resize(frame_right, (400, 600))
    combined_frame = np.concatenate((frame_left, frame_right), axis=1)
    cv2.line(combined_frame, (combined_frame.shape[1]//2, 0), (combined_frame.shape[1]//2, combined_frame.shape[0]), (0, 255, 0), 2)

    # Calculate positions for text
    left_text_position = (combined_frame.shape[1] // 4 - 100, combined_frame.shape[0] // 2)
    right_text_position = (3 * combined_frame.shape[1] // 4 - 100, combined_frame.shape[0] // 2)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('f'): 
        if Card is not None and Card.warp is not None:
            timestamp = time.strftime("%Y%m%d-%H%M%S")
            save_flattened_image(Card.warp, f"flattened_image_{timestamp}.png")
    elif key == ord('r'):
        KartuTerdeteksi_left.clear()
        KartuTerdeteksi_right.clear()
        GAME_STATUS = "Playing"
        WINNER = None
        game_ended = False
        JumlahKartuPemain = 0  # Reset card counts after the game ends
        JumlahKartuKomputer = 0
    elif key == ord('a'):  # Add all detected cards to left player's array
        KartuTerdeteksi_left.extend(new_cards_left)
        player_turn_taken = True
        JumlahKartuPemain += len(new_cards_left)
    elif key == ord('b'):  # Add all detected cards to right player's array
        KartuTerdeteksi_right.extend(new_cards_right)

    NilaiKiri = sum(value for _, value in KartuTerdeteksi_left)
    JumlahKanan = sum(value for _, value in KartuTerdeteksi_right)
    
    if KartuTerdeteksi_left:
            last_card_left, _ = KartuTerdeteksi_left[-1]
            cv2.putText(combined_frame, last_card_left, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(combined_frame, f"Total Left: {NilaiKiri}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    if KartuTerdeteksi_right:
            last_card_right, _ = KartuTerdeteksi_right[-1]
            cv2.putText(combined_frame, last_card_right, (combined_frame.shape[1]//2 + 10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(combined_frame, f"Total Right: {JumlahKanan}", (combined_frame.shape[1]//2 + 10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display game status
    cv2.putText(combined_frame, f"Game Status: {GAME_STATUS}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    if not game_ended and player_turn_taken:
        ai_played = computer_play(player_turn_taken)
        if ai_played:
            player_turn_taken = False 
            JumlahKartuKomputer += 1

        NilaiKiri = sum(value for _, value in KartuTerdeteksi_left)
        JumlahKanan = sum(value for _, value in KartuTerdeteksi_right)
    # Display the last detected card and total sum for each player
        
        GAME_STATUS, WINNER = play_blackjack(NilaiKiri, JumlahKanan, JumlahKartuPemain, JumlahKartuKomputer, GAME_STATUS)

        if WINNER:
            game_ended = True

    if game_ended:
        if WINNER == "Left Player Wins":
            cv2.putText(combined_frame, "Winner", left_text_position, cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)
            cv2.putText(combined_frame, "Loser", right_text_position, cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
        else:
            cv2.putText(combined_frame, "Loser", left_text_position, cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
            cv2.putText(combined_frame, "Winner", right_text_position, cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)
    
    cv2.imshow('Deteksi Kartu', combined_frame)
camera.release()
cv2.destroyAllWindows()



1/1 [==============================] - 0s 15ms/step


NameError: name 'NilaiKiri' is not defined

In [2]:
LabelKartu2s = [
    "2 Hearts", "3 Hearts", "4 Hearts", "5 Hearts", "6 Hearts", "7 Hearts", "8 Hearts", "9 Hearts", "10 Hearts", "Jack Hearts", "Queen Hearts", "King Hearts", "Ace Hearts",
    "2 Spades", "3 Spades", "4 Spades", "5 Spades", "6 Spades", "7 Spades", "8 Spades", "9 Spades", "10 Spades", "Jack Spades", "Queen Spades", "King Spades", "Ace Spades",
    "2 Diamonds", "3 Diamonds", "4 Diamonds", "5 Diamonds", "6 Diamonds", "7 Diamonds", "8 Diamonds", "9 Diamonds", "10 Diamonds", "Jack Diamonds", "Queen Diamonds", "King Diamonds", "Ace Diamonds",
    "2 Clubs", "3 Clubs", "4 Clubs", "5 Clubs", "6 Clubs", "7 Clubs", "8 Clubs", "9 Clubs", "10 Clubs", "Jack Clubs", "Queen Clubs", "King Clubs", "Ace Clubs"
]


In [2]:
import numpy as np
import cv2
import time 
from keras.models import load_model

model = load_model(r'C:\Users\HP\Documents\PCV\Poker2.h5')
camera = cv2.VideoCapture(1)
camera.set(10, 500)

card_min_area = 25000  
card_max_area = 45000  
card_count = 0  
last_time_processed = 0
Card = None
KartuTerdeteksi = []

NilaiKartu = {
    "2 Hearts": 2, "3 Hearts": 3, "4 Hearts": 4, "5 Hearts": 5, "6 Hearts": 6, "7 Hearts": 7, "8 Hearts": 8, "9 Hearts": 9, "10 Hearts": 10, 
    "Jack Hearts": 10, "Queen Hearts": 10, "King Hearts": 10, "Ace Hearts": 11, "2 Spades": 2, "3 Spades": 3, "4 Spades": 4, "5 Spades": 5, 
    "6 Spades" : 6, "7 Spades" : 7, "8 Spades" : 8, "9 Spades" : 9, "10 Spades" : 10, "Jack Spades" : 10, "Queen Spades" : 10, "King Spades" : 10, 
    "Ace Spades" : 11, "2 Diamonds" : 2, "3 Diamonds" : 3, "4 Diamonds" : 4, "5 Diamonds" : 5, "6 Diamonds" : 6, "7 Diamonds" : 7, "8 Diamonds" : 8,
    "9 Diamonds" : 9, "10 Diamonds" : 10, "Jack Diamonds" : 10, "Queen Diamonds" : 10, "King Diamonds" : 10, "Ace Diamonds" : 11, "2 Clubs" : 2, 
    "3 Clubs" : 3, "4 Clubs" : 4, "5 Clubs" : 5, "6 Clubs" : 6, "7 Clubs" : 7, "8 Clubs" : 8, "9 Clubs" : 9, "10 Clubs" : 10, "Jack Clubs" :10,
      "Queen Clubs" : 10, "King Clubs" : 10, "Ace Clubs" : 11
}


RANK_WIDTH = 70
RANK_HEIGHT = 125

SUIT_WIDTH = 70
SUIT_HEIGHT = 100

BKG_THRESH = 70
CARD_THRESH = 40

CORNER_WIDTH = 32
CORNER_HEIGHT = 84

player_hand = []
player_score = 0
game_over = False

def calculate_score(cards):
    score = sum(NilaiKartu[card] for card in cards)
    # Adjust for Aces
    for card in cards:
        if card.startswith("Ace") and score > 21:
            score -= 10  # Adjust Ace from 11 to 1
    return score

class Kartu:

    def __init__(self):
        self.contour = [] # Contour of card
        self.width, self.height = 0, 0 # Width and height of card
        self.corner_pts = [] # Corner points of card
        self.center = [] # Center point of card
        self.warp = [] # 200x300, flattened, grayed, blurred image
        self.rank_img = [] # Thresholded, sized image of card's rank
        self.suit_img = [] # Thresholded, sized image of card's suit
        self.best_rank_match = "Unknown" # Best matched rank
        self.best_suit_match = "Unknown" # Best matched suit
        self.rank_diff = 0 # Difference between rank image and best matched train rank image
        self.suit_diff = 0 # Difference between suit image and best matched train suit image
def ProsesFrame(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    img_w, img_h = np.shape(image)[:2]
    bkg_level = gray[int(img_h/100)][int(img_w/2)]
    thresh_level = bkg_level + BKG_THRESH

    _, thresh = cv2.threshold(blur, thresh_level, 255, cv2.THRESH_BINARY)

    edges = cv2.Canny(blur, threshold1=50, threshold2=150)

    # Combine threshold and edges (optional: based on your use case)
    combined = cv2.bitwise_or(thresh, edges)

    return combined
def ProsesKartu(contour, image):

    Card = Kartu()
    Card.contour = contour

    # Find perimeter of card and use it to approximate corner points
    peri = cv2.arcLength(contour,True)
    approx = cv2.approxPolyDP(contour,0.01*peri,True)

    if len(approx) != 4:
        return None
    
    pts = np.float32(approx)
    Card.corner_pts = pts

    # Find width and height of card's bounding rectangle
    x,y,w,h = cv2.boundingRect(contour)
    Card.width, Card.height = w, h

    # Warp card into 200x300 flattened image using perspective transform
    Card.warp = flattener(image, pts, w, h)
    return Card

def flattener(image, pts, w, h):
    s = np.sum(pts, axis = 2)
    diff = np.diff(pts, axis = -1)
    rect = np.zeros((4, 2), dtype = "float32")

    rect[0] = pts[np.argmin(s)]        # Top-left point
    rect[2] = pts[np.argmax(s)]        # Bottom-right point
    rect[1] = pts[np.argmin(diff)]     # Top-right point
    rect[3] = pts[np.argmax(diff)]     # Bottom-left point

    # Perspective transformation
    maxWidth, maxHeight = 200, 300
    dst = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warp = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
        

    return warp

def ProsesInputModel(image):
    processed_image = cv2.resize(image, (200, 300))
    # if len(processed_image.shape) == 2:  
    #     processed_image = cv2.cvtColor(processed_image, cv2.COLOR_GRAY2BGR)
        
    rgb = cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB)
    processed_image = np.asarray(rgb) / 255.0
    # processed_image = processed_image.astype('float32')
    return processed_image, rgb

def save_flattened_image(image, filename):
    if image is not None:
        cv2.imwrite(filename, image)

def Deteksi(frame):
    global Card
    LabelKartu2 = None
    combined = ProsesFrame(frame)
    contours, _ = cv2.findContours(combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    cv2.imshow('Threshold', combined)

    for contour in contours:
        area = cv2.contourArea(contour)
        if card_min_area < area < card_max_area:
            
            cv2.drawContours(frame, [contour], 0, (255, 0, 0), 2)
            Card = ProsesKartu(contour, frame)
            
            if Card is not None:
                processed_card_image, rgb = ProsesInputModel(Card.warp)
                prediction = model.predict(np.expand_dims(processed_card_image, axis=0))
                predicted_class = np.argmax(prediction, axis=1)

                cv2.imshow('Flattened', Card.warp)

                LabelKartu = [
    "2 Hearts", "3 Hearts", "4 Hearts", "5 Hearts", "6 Hearts", "7 Hearts", "8 Hearts", "9 Hearts", "10 Hearts", "Jack Hearts", "Queen Hearts", "King Hearts", "Ace Hearts",
    "2 Spades", "3 Spades", "4 Spades", "5 Spades", "6 Spades", "7 Spades", "8 Spades", "9 Spades", "10 Spades", "Jack Spades", "Queen Spades", "King Spades", "Ace Spades",
    "2 Diamonds", "3 Diamonds", "4 Diamonds", "5 Diamonds", "6 Diamonds", "7 Diamonds", "8 Diamonds", "9 Diamonds", "10 Diamonds", "Jack Diamonds", "Queen Diamonds", "King Diamonds", "Ace Diamonds",
    "2 Clubs", "3 Clubs", "4 Clubs", "5 Clubs", "6 Clubs", "7 Clubs", "8 Clubs", "9 Clubs", "10 Clubs", "Jack Clubs", "Queen Clubs", "King Clubs", "Ace Clubs"
]
                LabelPred = LabelKartu[predicted_class[0]]
                LabelKartu2 = LabelPred
                x,y,w,h = cv2.boundingRect(contour)
                cv2.rectangle(frame,(x,y),(x+w, y+h),(0,255,0),2)
                cv2.putText(frame,LabelPred,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5, (0,255,0),2)
        
    return frame, LabelKartu2
while camera.isOpened():
    ret, frame = camera.read()
    if not ret:
        continue

    frame, LabelTerdeteksi = Deteksi(frame)
    SkorTotal = sum(NilaiKartu[LabelKartu2] for LabelKartu2, _ in KartuTerdeteksi)
    y_offset = 30
    SkorTotal = 0

    for LabelKartu2, NilaiKartuTerdeteksi in KartuTerdeteksi:
        cv2.putText(frame, f"{LabelKartu2} (Nilai: {NilaiKartuTerdeteksi})", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        y_offset += 30  
        SkorTotal += NilaiKartuTerdeteksi
    cv2.putText(frame, f"Hand: {', '.join(player_hand)}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"Score: {player_score}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"Jumlah Nilai Kartu: {SkorTotal}", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    if player_score > 21:
        cv2.putText(frame, "Bust!", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        game_over = True
    elif player_score == 21:
        cv2.putText(frame, "Blackjack!", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        game_over = True

    cv2.imshow('Deteksi Kartu', frame)
    if not game_over:
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('f'): 
            if Card is not None and Card.warp is not None:
                timestamp = time.strftime("%Y%m%d-%H%M%S")
                save_flattened_image(Card.warp, f"flattened_image_{timestamp}.png")
        elif key == ord('a'): 
            if LabelTerdeteksi:
                NilaiKartuTerdeteksi = NilaiKartu.get(LabelTerdeteksi, 0)
                KartuTerdeteksi.append((LabelTerdeteksi, NilaiKartuTerdeteksi))
        elif key == ord('r'):
            KartuTerdeteksi.clear()
        elif key == ord('h'):  # Hit: Take another card
            if LabelTerdeteksi:
                player_hand.append(LabelTerdeteksi)
                player_score = calculate_score(player_hand)
        elif key == ord('s'):  # Stand: End turn
            game_over = True
    else:
        # Game over, wait for 'n' to start a new game or 'q' to quit
        cv2.putText(frame, "Game Over. Press 'n' for New Game, 'q' to Quit", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        key = cv2.waitKey(0)  # Wait indefinitely for a key press
        if key == ord('n'):
            game_over = False
            player_hand = []
            player_score = 0
            # Reset other game states if necessary
        elif key == ord('q'):
            break

camera.release()
cv2.destroyAllWindows()



1/1 [==============================] - 0s 20ms/step
